In [ ]:
import pickle
import random
from typing import List, Set
import matplotlib.pyplot as plt
from settings import VEHICLE_NUM, VERTICES_NUM, POPULATION_SIZE, MUTATION_RATE, GENERATION_NUM

Point = tuple[int, int]
vertices: List[Point] = [(0, 0)]
distance_matrix: List[List[float]] = []
population: List['Chromosome'] = []
compare_population: List['Chromosome'] = []


class Chromosome:
    def __init__(self):
        self.genes: List[int] = []
        self.fitness: float

    def __eq__(self, __o: object) -> bool:
        return self.genes == __o.genes

    def correction(self, pos: int) -> 'Chromosome':
        temp = pos
        while self.genes[temp] == -1:
            temp -= 1
        if temp % VEHICLE_NUM > pos:
            self.genes.insert(pos, self.genes.pop(temp))
        else:
            self.genes.insert(pos-1, self.genes.pop(temp))
        return self

    def calc_fitness(self) -> float:
        max: float = 0
        prev = 0
        sum = 0
        for i, current in enumerate(self.genes):
            if current == -1:
                if sum > max:
                    max = sum
                if sum == 0:
                    return self.correction(i).calc_fitness()
                sum = 0
                prev = 0
                continue
            sum += distance_matrix[current][prev]
            prev = current
        self.fitness = 1/max
        return 1/max

    def mutate(self, rate=MUTATION_RATE) -> 'Chromosome':
        if random.randint(1, 100) <= rate:
            pos1 = random.randint(0, len(self.genes)-2)
            pos2 = random.randint(0, len(self.genes)-2)
            self.genes[pos1], self.genes[pos2] = self.genes[pos2], self.genes[pos1]
        return self

    def random(self) -> 'Chromosome':
        order: List[int] = [i+1 for i in range(VERTICES_NUM)]
        for i in range(VEHICLE_NUM-1):
            order.append(-1)
        random.shuffle(order)
        order.append(-1)
        self.genes += order
        self.calc_fitness()
        return self


def generate_population():
    for i in range(POPULATION_SIZE):
        population.append(Chromosome().random())
    population.sort(key=lambda chromosome: chromosome.fitness)


def rw_selection(p: List['Chromosome']) -> 'Chromosome':
    max: float = sum(chromosome.fitness for chromosome in p)
    pick = random.uniform(0, max)
    current = 0
    for chromosome in p:
        current += chromosome.fitness
        if current > pick:
            return chromosome


def crossover(parent_1: 'Chromosome', parent_2: 'Chromosome') -> 'Chromosome':
    temp = random.randint(0, VEHICLE_NUM-2)
    cut_point = [i for i, v in enumerate(
        parent_1.genes) if v == -1][temp]
    remain = VEHICLE_NUM - temp - 2
    child = parent_1.genes[:cut_point+1]
    for gene in parent_2.genes:
        if gene != -1 and gene not in child:
            child.append(gene)
        if gene == -1 and remain > 0:
            child.append(-1)
            remain -= 1
    child.append(-1)
    for_return = Chromosome()
    for_return.genes = child
    for_return.mutate()
    for_return.calc_fitness()
    return for_return


with open('vertices', 'rb') as vt:
    vertices = pickle.load(vt)
with open('distance', 'rb') as dt:
    distance_matrix = pickle.load(dt)
generate_population()
# compare_population = population
solutions: Set[str] = set()
# compare_solutions: Set[str] = set()
random_solution: float = Chromosome().random().fitness

visited_ax= []
random_ax = []
ga_ax = []

for i in range(GENERATION_NUM):
    solutions.update(str(c.genes) for c in population)
    # compare_solutions.update(str(c.genes) for c in compare_population)
    population = population[POPULATION_SIZE//2:]
    # compare_population = compare_population[POPULATION_SIZE//2:]
    while len(population) < POPULATION_SIZE:
        population.append(crossover(rw_selection(population),
                          rw_selection(population)))
        # compare_population.append(crossover(rw_selection(
        #     compare_population), rw_selection(compare_population)))
        random_solution = max(random_solution, Chromosome().random().fitness)
    population.sort(key=lambda chromosome: chromosome.fitness)
    # compare_population.sort(key=lambda chromosome: chromosome.fitness)
    print(f'generation {i}')
    print(
        f'solution {len(solutions)} - {POPULATION_SIZE+i*POPULATION_SIZE/2}')
    print(
        f'{1/population[-1].fitness} - {1/random_solution}')
    visited_ax.append(POPULATION_SIZE+i*POPULATION_SIZE/2)
    random_ax.append(1/random_solution)
    ga_ax.append(1/1/population[-1].fitness)
print(population[-1].genes)

In [ ]:
plt.plot(visited_ax,ga_ax, label = 'Giải thuật di truyền')
plt.plot(visited_ax,random_ax, label = 'Ngẫu nhiên')
plt.xlabel('Số lời giải')
plt.ylabel('Quảng đường')
plt.title(f'O({VEHICLE_NUM+VERTICES_NUM-1}!)')
plt.legend()